# Basic face recognizer using pre trained model

Below the is the implementation of a very basic face recognizer which can identify the face of the person showing on a web cam.
The implementation is inspired by two path breaking papers on facial recognition using deep convoluted neural network, namely FaceNet and DeepFace.

I have used pre trained model Keras-OpenFace which is an open source Keras implementation of the OpenFace (Originally Torch implemented) 

The pretrained model that I have used is by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.

# Import the prerequisite libraries

We will be importing utils.py from https://github.com/iwantooxxoox/Keras-OpenFace/blob/master/utils.py (available with code) which contains utility functions to create the neural network and load the weights assoiated with it.

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils

%load_ext autoreload
%autoreload 2

np.set_printoptions()

Using TensorFlow backend.


# Contructing the neural network model
The model here constructed is based on FaceNet's Inception model.

The implementation of model is available at: https://github.com/iwantooxxoox/Keras-OpenFace

In [2]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

# Loading the model with pretrained weights

FaceNet is trained by minimizing the triplet loss. I will be  loading a previously trained model. weights are avaiable at https://github.com/iwantooxxoox/Keras-OpenFace in the "weights" folder which is also provided in this source.

This can take a couple of minutes to execute and depends on the speed of your machine.


In [3]:
# Load weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

## About <font color=blue>image_to_embedding</font> function        
When the model is loaded with pre trained weights, then we can create the **128 dimensional embedding vectors** for all the face images stored in the "images" folder. **"image_to_embedding"** function pass an image to the Inception network to generate the embedding vector.

In [4]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


## About <font color=blue>recognize_face</font> function
This function calculate similarity between the captured image and the images that are already been stored. It passes the image to the trained neural network to generate its embedding vector. Which is then compared with all the embedding vectors of the images stored by calculating L2 Euclidean distance. 

If the minimum L2 distance between two embeddings is less than a threshpld (here I have taken the threashhold as .68 (which can be adjusted) then we have a match.

In [5]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.68:
        return str(name)
    else:
        return None

## About <font color=blue>create_input_image_embeddings</font> function
This function generates 128 dimensional image ebeddings of all the images stored in the "images" directory by feed forwarding the images to a trained neural network. It creates a dictionary with key as the name of the face and value as embedding


## About <font color=blue>recognize_faces_in_cam</font> function
This function capture image from the webcam, detect a face in it and crop the image to have a face only, which is then passed to recognize_face function. 

In [6]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    

    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Capturing the face image
Following code captures 10 face images of the person. They all are stored in **"images"** folder with the name User_1 to User_10. Select a good captured image from the set of 10 images. Rename it with the name of person and delete rest of them. This image will be used for recognizing the the identity of the person using one shot learning.

In [8]:
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

count = 0
while(True):
    ret, img = cam.read()
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("images/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 10: # Take 30 face sample and stop video
         break
cam.release()
cv2.destroyAllWindows()

# Lets run the face recognizer program :-)

In [7]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)


Euclidean distance from Sumantra is 1.2080387
Euclidean distance from User_8 is 1.2754925
Euclidean distance from User_9 is 1.2896584
Euclidean distance from User_2 is 1.3759464
Euclidean distance from User_3 is 1.3593597
Euclidean distance from User_1 is 1.3467709
Euclidean distance from User_4 is 1.3668903
Euclidean distance from User_5 is 1.2896168
Euclidean distance from User_7 is 1.3124069
Euclidean distance from User_6 is 1.3128872
Euclidean distance from Kunal is 1.4443932
Euclidean distance from User_10 is 1.3382477
Euclidean distance from Sumantra is 1.0309616
Euclidean distance from User_8 is 1.1428874
Euclidean distance from User_9 is 1.1832097
Euclidean distance from User_2 is 1.190028
Euclidean distance from User_3 is 1.1332326
Euclidean distance from User_1 is 1.0593026
Euclidean distance from User_4 is 1.1239827
Euclidean distance from User_5 is 1.1556511
Euclidean distance from User_7 is 1.0821204
Euclidean distance from User_6 is 1.0671164
Euclidean distance from Kunal

Euclidean distance from Sumantra is 0.79420006
Euclidean distance from User_8 is 0.8084164
Euclidean distance from User_9 is 0.8123726
Euclidean distance from User_2 is 0.76642966
Euclidean distance from User_3 is 0.7450571
Euclidean distance from User_1 is 0.81018806
Euclidean distance from User_4 is 0.7288532
Euclidean distance from User_5 is 0.7519123
Euclidean distance from User_7 is 0.7053392
Euclidean distance from User_6 is 0.8096682
Euclidean distance from Kunal is 1.0344887
Euclidean distance from User_10 is 0.767903
Euclidean distance from Sumantra is 0.74351954
Euclidean distance from User_8 is 0.88843656
Euclidean distance from User_9 is 0.8811774
Euclidean distance from User_2 is 0.8351943
Euclidean distance from User_3 is 0.7691329
Euclidean distance from User_1 is 0.9036379
Euclidean distance from User_4 is 0.8001303
Euclidean distance from User_5 is 0.84623915
Euclidean distance from User_7 is 0.77987266
Euclidean distance from User_6 is 0.8701673
Euclidean distance fro

Euclidean distance from Sumantra is 0.8054829
Euclidean distance from User_8 is 0.8402925
Euclidean distance from User_9 is 0.7882025
Euclidean distance from User_2 is 0.7171133
Euclidean distance from User_3 is 0.6710346
Euclidean distance from User_1 is 0.7410046
Euclidean distance from User_4 is 0.679932
Euclidean distance from User_5 is 0.76404905
Euclidean distance from User_7 is 0.67787087
Euclidean distance from User_6 is 0.78096557
Euclidean distance from Kunal is 0.96330637
Euclidean distance from User_10 is 0.66786045
Euclidean distance from Sumantra is 0.79738134
Euclidean distance from User_8 is 0.7237061
Euclidean distance from User_9 is 0.6888621
Euclidean distance from User_2 is 0.6256561
Euclidean distance from User_3 is 0.5997569
Euclidean distance from User_1 is 0.66785043
Euclidean distance from User_4 is 0.6046625
Euclidean distance from User_5 is 0.63334984
Euclidean distance from User_7 is 0.56473786
Euclidean distance from User_6 is 0.6742055
Euclidean distance f

Euclidean distance from Sumantra is 0.6045843
Euclidean distance from User_8 is 0.5891217
Euclidean distance from User_9 is 0.51347345
Euclidean distance from User_2 is 0.6336046
Euclidean distance from User_3 is 0.55869776
Euclidean distance from User_1 is 0.8417545
Euclidean distance from User_4 is 0.69869006
Euclidean distance from User_5 is 0.5689574
Euclidean distance from User_7 is 0.59087217
Euclidean distance from User_6 is 0.5650375
Euclidean distance from Kunal is 0.94354546
Euclidean distance from User_10 is 0.4844034
Euclidean distance from Sumantra is 0.66445994
Euclidean distance from User_8 is 0.64321315
Euclidean distance from User_9 is 0.548579
Euclidean distance from User_2 is 0.67532134
Euclidean distance from User_3 is 0.634257
Euclidean distance from User_1 is 0.88245773
Euclidean distance from User_4 is 0.76720166
Euclidean distance from User_5 is 0.63099194
Euclidean distance from User_7 is 0.65047765
Euclidean distance from User_6 is 0.62492675
Euclidean distanc

Euclidean distance from Sumantra is 0.6873001
Euclidean distance from User_8 is 0.720047
Euclidean distance from User_9 is 0.57310617
Euclidean distance from User_2 is 0.6073069
Euclidean distance from User_3 is 0.58341813
Euclidean distance from User_1 is 0.830996
Euclidean distance from User_4 is 0.6666758
Euclidean distance from User_5 is 0.6428801
Euclidean distance from User_7 is 0.6067213
Euclidean distance from User_6 is 0.6851651
Euclidean distance from Kunal is 0.98102665
Euclidean distance from User_10 is 0.47584537
Euclidean distance from Sumantra is 0.5980543
Euclidean distance from User_8 is 0.77282935
Euclidean distance from User_9 is 0.6918731
Euclidean distance from User_2 is 0.72855777
Euclidean distance from User_3 is 0.5958465
Euclidean distance from User_1 is 0.9069025
Euclidean distance from User_4 is 0.7734351
Euclidean distance from User_5 is 0.7447889
Euclidean distance from User_7 is 0.72125983
Euclidean distance from User_6 is 0.6999717
Euclidean distance from

...

Euclidean distance from Sumantra is 0.7848069
Euclidean distance from User_8 is 0.660292
Euclidean distance from User_9 is 0.6622154
Euclidean distance from User_2 is 0.63472474
Euclidean distance from User_3 is 0.62404346
Euclidean distance from User_1 is 0.8596554
Euclidean distance from User_4 is 0.78547674
Euclidean distance from User_5 is 0.74578714
Euclidean distance from User_7 is 0.72632754
Euclidean distance from User_6 is 0.66967803
Euclidean distance from Kunal is 0.92251825
Euclidean distance from User_10 is 0.5203215
Euclidean distance from Sumantra is 0.64463675
Euclidean distance from User_8 is 0.78430074
Euclidean distance from User_9 is 0.7010065
Euclidean distance from User_2 is 0.6677352
Euclidean distance from User_3 is 0.5880523
Euclidean distance from User_1 is 0.9127981
Euclidean distance from User_4 is 0.7662181
Euclidean distance from User_5 is 0.8103388
Euclidean distance from User_7 is 0.7425183
Euclidean distance from User_6 is 0.73501724
Euclidean distance 

Euclidean distance from Sumantra is 0.9599359
Euclidean distance from User_8 is 1.1534847
Euclidean distance from User_9 is 1.1983472
Euclidean distance from User_2 is 1.1637144
Euclidean distance from User_3 is 1.0645044
Euclidean distance from User_1 is 1.2488736
Euclidean distance from User_4 is 1.2153953
Euclidean distance from User_5 is 1.2196053
Euclidean distance from User_7 is 1.1861662
Euclidean distance from User_6 is 1.1445401
Euclidean distance from Kunal is 1.3203232
Euclidean distance from User_10 is 0.96564794
Euclidean distance from Sumantra is 0.9450796
Euclidean distance from User_8 is 1.1510704
Euclidean distance from User_9 is 1.1893206
Euclidean distance from User_2 is 1.1516987
Euclidean distance from User_3 is 1.0346589
Euclidean distance from User_1 is 1.2405051
Euclidean distance from User_4 is 1.1931503
Euclidean distance from User_5 is 1.208693
Euclidean distance from User_7 is 1.1828032
Euclidean distance from User_6 is 1.1441293
Euclidean distance from Kuna

Euclidean distance from Sumantra is 0.7189082
Euclidean distance from User_8 is 0.9196781
Euclidean distance from User_9 is 0.86403614
Euclidean distance from User_2 is 0.84956306
Euclidean distance from User_3 is 0.7635689
Euclidean distance from User_1 is 1.0132582
Euclidean distance from User_4 is 0.9034248
Euclidean distance from User_5 is 0.94226986
Euclidean distance from User_7 is 0.89373046
Euclidean distance from User_6 is 0.924999
Euclidean distance from Kunal is 1.112133
Euclidean distance from User_10 is 0.59987473
Euclidean distance from Sumantra is 0.7679806
Euclidean distance from User_8 is 0.96593463
Euclidean distance from User_9 is 0.9628898
Euclidean distance from User_2 is 0.9341009
Euclidean distance from User_3 is 0.8286008
Euclidean distance from User_1 is 1.0244938
Euclidean distance from User_4 is 0.9722099
Euclidean distance from User_5 is 1.0057135
Euclidean distance from User_7 is 0.9606597
Euclidean distance from User_6 is 0.9637291
Euclidean distance from 

Euclidean distance from Sumantra is 0.7945989
Euclidean distance from User_8 is 1.0348923
Euclidean distance from User_9 is 1.0409796
Euclidean distance from User_2 is 0.93794644
Euclidean distance from User_3 is 0.8399421
Euclidean distance from User_1 is 1.0416346
Euclidean distance from User_4 is 0.9368342
Euclidean distance from User_5 is 1.0707917
Euclidean distance from User_7 is 0.98148274
Euclidean distance from User_6 is 1.0150347
Euclidean distance from Kunal is 1.122068
Euclidean distance from User_10 is 0.7832676
Euclidean distance from Sumantra is 0.8057634
Euclidean distance from User_8 is 1.0597832
Euclidean distance from User_9 is 1.048977
Euclidean distance from User_2 is 1.0207813
Euclidean distance from User_3 is 0.90973914
Euclidean distance from User_1 is 1.1701661
Euclidean distance from User_4 is 1.0796034
Euclidean distance from User_5 is 1.1159945
Euclidean distance from User_7 is 1.0792553
Euclidean distance from User_6 is 1.0671304
Euclidean distance from Kun

Euclidean distance from Sumantra is 1.3251995
Euclidean distance from User_8 is 1.1971623
Euclidean distance from User_9 is 1.2384392
Euclidean distance from User_2 is 1.1774195
Euclidean distance from User_3 is 1.2261913
Euclidean distance from User_1 is 1.00804
Euclidean distance from User_4 is 1.0939779
Euclidean distance from User_5 is 1.1790615
Euclidean distance from User_7 is 1.1169106
Euclidean distance from User_6 is 1.1097476
Euclidean distance from Kunal is 1.0394988
Euclidean distance from User_10 is 1.2747198
Euclidean distance from Sumantra is 0.58890325
Euclidean distance from User_8 is 0.722752
Euclidean distance from User_9 is 0.6653603
Euclidean distance from User_2 is 0.63710713
Euclidean distance from User_3 is 0.6192552
Euclidean distance from User_1 is 0.8298574
Euclidean distance from User_4 is 0.67715985
Euclidean distance from User_5 is 0.7171622
Euclidean distance from User_7 is 0.626944
Euclidean distance from User_6 is 0.70045215
Euclidean distance from Kuna